In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim import corpora, models
# from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
import altair as alt
from altair_saver import save
import os
import re
from nltk.util import ngrams
from itertools import chain

import numpy as np
import warnings
warnings.simplefilter("ignore")

C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
df = pd.read_csv('Data_mesh_publications_cleaned.csv')
df_sample = df. copy()
df

,Filename,Content,Publisher,Year,Publication_type
0,Breaking Down Data Silos Data Mesh to Achieve ...,abstract data localization law becoming make h...,IEEE,2023,conference
1,Decentralized Data Governance as Part of a Dat...,abstract data socio technical decentralized an...,IEEE,2023,conference
2,Enterprise Data Strategy A Decentralized Data ...,abstract enterprise experience exponential gro...,IEEE,2022,conference
3,Finding Your Way Through the Jungle of Big Dat...,abstract paper present systematic ofcommon ana...,IEEE,2021,conference
4,CoK A Survey of Privacy Challenges in Relation...,growing volume data appear multiple dis tribut...,Springer,2022,conference
5,Converging Data Mesh and Microservice Principl...,company invested driven design sup port data d...,Springer,2023,conference
6,Data Integration Revitalized From Data Warehou...,year data integration architecture evolved fro...,Springer,2023,conference
7,Data Mesh as Distributed Data Platform for Lar...,rapid increase data volume last decade promote...,Springer,2023,conference
8,Data Fabric and Data Mesh Approaches with AI,look back data architecture developed response...,Springer,2023,book
9,Cow Mesh a data-mesh architecture to unify dai...,dairy economically signi cant industry caters ...,ResearchGate,2023,journal


# TFIDF

In [3]:
# Define a function to preprocess text data
def preprocess_text(text):
    # Remove non-alphanumeric characters and special characters
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text)

    return text

# Preprocess the PDF content before fitting it to the TF-IDF vectorizer
df['Content'] = df['Content'].apply(preprocess_text)

In [4]:
# def find_top_words_for_each_document(df, num_top_words=5):

#     # Create a TF-IDF vectorizer
#     tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2, 4), token_pattern=r'\b\w[\w-]+\b')

#     # Fit and transform the PDF content
#     tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])

#     # Get feature names (words) from the TF-IDF vectorizer
#     feature_names = tfidf_vectorizer.get_feature_names_out()

#     # Create a DataFrame with TF-IDF values
#     tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

#     # Now, for each document, find the top words based on TF-IDF scores
#     top_words_for_each_document = []
#     for i, row in enumerate(tfidf_df.iterrows()):
#         _, document_tfidf_scores = row
#         top_words_index = document_tfidf_scores.argsort()[-num_top_words:][::-1]
#         top_words = [(feature_names[index], document_tfidf_scores[index]) for index in top_words_index]
#         top_words_for_each_document.append(top_words)

#     # Create a new DataFrame with 'pdf_content' and 'top_words'
#     result_df = pd.DataFrame({'Filename': df['Filename'],'Year': df['Year'],'Publisher': df['Publisher'],'Publication_type': df['Publication_type'], 'Content': df['Content'], 'top_words': top_words_for_each_document})

#     return result_df

In [5]:
def find_top_words_for_each_document(df, num_top_words=5):
    df['Content'] = df['Content'].apply(preprocess_text)

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(2, 4), token_pattern=r'\b\w[\w-]+\b')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Content'])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Correcting split words in feature names
    corrected_features = {name: name.replace('arti cial', 'artificial') for name in feature_names}
    corrected_feature_names = [corrected_features[name] for name in feature_names]

    # Create a DataFrame with corrected feature names and TF-IDF values
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=corrected_feature_names)

    # Now, for each document, find the top words based on corrected TF-IDF scores
    top_words_for_each_document = []
    for i, row in tfidf_df.iterrows():
        top_words_index = row.argsort()[-num_top_words:][::-1]
        top_words = [(corrected_feature_names[index], row[index]) for index in top_words_index]
        top_words_for_each_document.append(top_words)

    # Create a new DataFrame with results
    result_df = pd.DataFrame({
        'Filename': df['Filename'],
        'Year': df['Year'],
        'Publisher': df['Publisher'],
        'Publication_type': df['Publication_type'],
        'Content': df['Content'],
        'top_words': top_words_for_each_document
    })

    return result_df


In [6]:
# Call the function
result_df_tfidf = find_top_words_for_each_document(df)
#result_df_tfidf

# Results with filename

In [7]:
result_data_tfidf_filename = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Filename = row['Filename']
    top_words = row['top_words']

    # Iterate through each tuple in the list of top_words
    for word, tfidf in top_words:
        result_data_tfidf_filename.append({'Filename': Filename, 'topic_word': word, 'tfidf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_filename = pd.DataFrame(result_data_tfidf_filename)

# result_df_tfidf_filename

In [8]:
import pandas as pd

# Assuming result_df_3 is your DataFrame
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None) 
# pd.set_option('display.max_colwidth', None)# Show all rows

# Display the entire DataFrame
result_df_tfidf_filename


,Filename,topic_word,tfidf_value
0,Breaking Down Data Silos Data Mesh to Achieve ...,data localization,0.642871
1,Breaking Down Data Silos Data Mesh to Achieve ...,data aggregation,0.077282
2,Breaking Down Data Silos Data Mesh to Achieve ...,potential benefit,0.068927
3,Breaking Down Data Silos Data Mesh to Achieve ...,localization data,0.065599
4,Breaking Down Data Silos Data Mesh to Achieve ...,data localization data,0.062975
...,...,...,...
100,Utilization of Data Mesh Framework as a Part o...,driven design,0.086392
101,Utilization of Data Mesh Framework as a Part o...,software development,0.072312
102,Utilization of Data Mesh Framework as a Part o...,data product,0.068521
103,Utilization of Data Mesh Framework as a Part o...,data engineering,0.063784


In [9]:
# Select the top terms for each PDF
top_terms_df_tfidf = result_df_tfidf_filename.groupby('Filename').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf['tfidf_value'] = top_terms_df_tfidf['tfidf_value'] + np.random.rand(top_terms_df_tfidf.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_tfidf).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Filename:N',
    color=alt.Color('tfidf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='topic_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tfidf_value", order="descending")],
    groupby=["Filename"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tfidf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='topic_word:N',
    color=alt.condition(alt.datum.tfidf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_tfidf = (heatmap + text).properties(width=1000)


In [10]:
# save(chart_tfidf, 'tfidf_heatmap.html')
chart_tfidf

alt.LayerChart(...)

# Result with publisher

In [11]:
result_data_tfidf_publisher = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publisher = row['Publisher']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publisher.append({'Publisher': Publisher, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publisher = pd.DataFrame(result_data_tfidf_publisher)


result_df_tfidf_publisher

,Publisher,top_word,tf_idf_value
0,IEEE,data localization,0.642871
1,IEEE,data aggregation,0.077282
2,IEEE,potential benefit,0.068927
3,IEEE,localization data,0.065599
4,IEEE,data localization data,0.062975
...,...,...,...
100,miscellaneous,driven design,0.086392
101,miscellaneous,software development,0.072312
102,miscellaneous,data product,0.068521
103,miscellaneous,data engineering,0.063784


In [12]:
result_df_tfidf_publisher.describe()

,tf_idf_value
count,105.000000
mean,0.115637
std,0.090302
min,0.044733
25%,0.069205
50%,0.087637
75%,0.119155
max,0.642871


In [13]:
# Select the top terms for each PDF
top_terms_df_publisher = result_df_tfidf_publisher.groupby('Publisher').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publisher['tf_idf_value'] = top_terms_df_publisher['tf_idf_value'] + np.random.rand(top_terms_df_publisher.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publisher).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publisher:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publisher"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publisher = (heatmap + text).properties(width=1300)
chart_publisher

alt.LayerChart(...)

In [14]:
# save(chart_tfidf_publisher_2, 'tfidf_heatmap_publisher.html')

# Results with Publisher

In [15]:
result_data_tfidf_publication_type = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Publication_type = row['Publication_type']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_publication_type.append({'Publication_type': Publication_type, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_df_tfidf_publication_type = pd.DataFrame(result_data_tfidf_publication_type)


result_df_tfidf_publication_type

,Publication_type,top_word,tf_idf_value
0,conference,data localization,0.642871
1,conference,data aggregation,0.077282
2,conference,potential benefit,0.068927
3,conference,localization data,0.065599
4,conference,data localization data,0.062975
...,...,...,...
100,master thesis,driven design,0.086392
101,master thesis,software development,0.072312
102,master thesis,data product,0.068521
103,master thesis,data engineering,0.063784


In [16]:
# Select the top terms for each PDF
top_terms_df_publication_type = result_df_tfidf_publication_type.groupby('Publication_type').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_publication_type['tf_idf_value'] = top_terms_df_publication_type['tf_idf_value'] + np.random.rand(top_terms_df_publication_type.shape[0]) * 0.0001

# Create a base chart
base = alt.Chart(top_terms_df_publication_type).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Publication_type:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Publication_type"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_publication_type = (heatmap + text).properties(width=1300)
chart_publication_type

alt.LayerChart(...)

In [17]:
result_data_tfidf_date = []

# Iterate through each row in the original DataFrame
for _, row in result_df_tfidf.iterrows():
    Year = row['Year']
    topic_words = row['top_words']

    # Iterate through each tuple in the list of topic_words
    for word, tfidf in topic_words:
        result_data_tfidf_date.append({'Year': Year, 'top_word': word, 'tf_idf_value': tfidf})

# Create the result DataFrame from the list
result_data_tfidf_date = pd.DataFrame(result_data_tfidf_date)


result_data_tfidf_date

,Year,top_word,tf_idf_value
0,2023,data localization,0.642871
1,2023,data aggregation,0.077282
2,2023,potential benefit,0.068927
3,2023,localization data,0.065599
4,2023,data localization data,0.062975
...,...,...,...
100,2021,driven design,0.086392
101,2021,software development,0.072312
102,2021,data product,0.068521
103,2021,data engineering,0.063784


In [18]:
# Select the top terms for each PDF
top_terms_df_tfidf_dates = result_data_tfidf_date.groupby('Year').head(5)

# Add a small random value to 'idf_value' for better visualization
top_terms_df_tfidf_dates['tf_idf_value'] = top_terms_df_tfidf_dates['tf_idf_value'] + np.random.rand(top_terms_df_tfidf_dates.shape[0]) * 0.0001

# Create a base chart
# Create a base chart
base = alt.Chart(top_terms_df_tfidf_dates).encode(
    x=alt.X('rank:O', axis=None),  # Use rank for x-axis
    y='Year:N',
    color=alt.Color('tf_idf_value:Q', scale=alt.Scale(scheme='viridis')),
    text='top_word:N'  # Corrected column name to 'topic_word'
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tf_idf_value", order="descending")],
    groupby=["Year"]
)

# Create a heatmap
heatmap = base.mark_rect().encode(
    color='tf_idf_value:Q',
)

# Create text labels with conditional color
text = base.mark_text(baseline='middle',fontWeight='bold').encode(
    text='top_word:N',
    color=alt.condition(alt.datum.tf_idf_value >= 0.4, alt.value('white'), alt.value('black'))
)

# Display the heatmap and text labels
chart_date_tfidf_type = (heatmap + text).properties(width=1300)
chart_date_tfidf_type

alt.LayerChart(...)